# 带工具的 BabyAGI

本笔记本基于 [baby agi](baby_agi.html)，但展示了如何替换执行链。之前的执行链只是一个会编造内容的 LLM。通过将其替换为具有访问工具权限的代理，我们有望获得真实可靠的信息。

## 安装和导入所需模块

In [1]:
from typing import Optional

from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_experimental.autonomous_agents import BabyAGI
from langchain_openai import OpenAI, OpenAIEmbeddings

## 连接到向量存储

根据你使用的向量存储的不同，这一步可能会有所不同。

In [2]:
%pip install faiss-cpu > /dev/null
%pip install google-search-results > /dev/null
from langchain.docstore import InMemoryDocstore
from langchain_community.vectorstores import FAISS

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# 定义你的嵌入模型
embeddings_model = OpenAIEmbeddings()
# 初始化空的向量存储
import faiss

embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})

## 定义链

BabyAGI 依赖于三个 LLM 链：
- 任务创建链，用于选择要添加到列表中的新任务
- 任务优先级链，用于重新设置任务优先级
- 执行链，用于执行任务


注意：在本笔记本中，执行链现在将是一个代理。

In [ ]:
from langchain.agents import AgentExecutor, Tool, ZeroShotAgent
from langchain.chains import LLMChain
from langchain_community.utilities import SerpAPIWrapper
from langchain_openai import OpenAI

todo_prompt = PromptTemplate.from_template(
    "你是一个规划专家，擅长为给定目标制定待办事项列表。请为以下目标制定待办事项列表：{objective}"
)
todo_chain = LLMChain(llm=OpenAI(temperature=0), prompt=todo_prompt)
search = SerpAPIWrapper()
tools = [
    Tool(
        name="搜索",
        func=search.run,
        description="当你需要回答关于当前事件的问题时非常有用",
    ),
    Tool(
        name="待办事项",
        func=todo_chain.run,
        description="当你需要制定待办事项列表时非常有用。输入：一个目标以创建待办事项列表。输出：该目标的待办事项列表。请明确目标是什么！",
    ),
]


prefix = """你是一个根据以下目标执行任务的 AI：{objective}。请考虑这些已完成的任务：{context}"""
suffix = """问题：{task}
{agent_scratchpad}"""
prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["objective", "task", "context", "agent_scratchpad"],
)

In [5]:
llm = OpenAI(temperature=0)
llm_chain = LLMChain(llm=llm, prompt=prompt)
tool_names = [tool.name for tool in tools]
agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True
)

### 运行 BabyAGI

现在是时候创建 BabyAGI 控制器并观察它如何努力实现你的目标了。

In [ ]:
目标 = "为今天的旧金山写一份天气报告"

In [ ]:
# LLMChains 的日志记录
verbose = False
# 如果为 None，将会无限继续
max_iterations: Optional[int] = 3
baby_agi = BabyAGI.from_llm(
    llm=llm,
    vectorstore=vectorstore,
    task_execution_chain=agent_executor,
    verbose=verbose,
    max_iterations=max_iterations,
)

In [ ]:
baby_agi({"objective": 目标})


*****TASK LIST*****

1: Make a todo list

*****NEXT TASK*****

1: Make a todo list


> Entering new AgentExecutor chain...
Thought: I need to come up with a todo list
Action: TODO
Action Input: Write a weather report for SF today

1. Research current weather conditions in San Francisco
2. Gather data on temperature, humidity, wind speed, and other relevant weather conditions
3. Analyze data to determine current weather trends
4. Write a brief introduction to the weather report
5. Describe current weather conditions in San Francisco
6. Discuss any upcoming weather changes
7. Summarize the weather report
8. Proofread and edit the report
9. Submit the report I now know the final answer
Final Answer: The todo list for writing a weather report for SF today is: 1. Research current weather conditions in San Francisco; 2. Gather data on temperature, humidity, wind speed, and other relevant weather conditions; 3. Analyze data to determine current weather trends; 4. Write a brief introduction t

{'objective': 'Write a weather report for SF today'}